In [4]:
import sys
sys.path.insert(0, '../../src/')

import pandas as pd
import auto_co2 as co2
import importlib
importlib.reload(co2)

#auth_file_path = ''
#df = co2.data.download_and_load_co2_data()
df = co2.data.load_co2_data(filepath='../../data/raw')
df = co2.data.dataviz_preprocess(df)
df['ElectricRange'] = df['ElectricRange'].fillna(0)
df['InnovativeEmissionsReductionWltp'] = df['InnovativeEmissionsReductionWltp'].fillna(0)
df['ElectricConsumption'] = df['ElectricConsumption'].fillna(0)
df = co2.data.standardize_innovtech(df)
df = df.drop(columns=['ID', 'Country', 'ManufacturerName', 'WltpTestMass', 'FuelConsumption', 'RegistrationDate', 'AxleWidthSteering', 'ManufNameOem', 'FuelMode', 'EngineCapacity', 'AxleWidthSteering'])



# Group by the specified columns and compute the mean for the other columns
counts = df.groupby(['CommercialName', 'FuelType', 'CategoryOf', 'InnovativeTechnology']).size()

# Group by the specified columns and compute the mean for the other columns
df = df.groupby(['CommercialName', 'FuelType', 'CategoryOf', 'InnovativeTechnology']).agg({
    'Pool': lambda x: x.mode()[0],
    'Co2EmissionsWltp': 'mean',
    'EnginePower': 'mean',
    'ElectricConsumption': 'mean',
    'InnovativeEmissionsReductionWltp': 'mean',
    'ElectricRange': 'mean',
    'MassRunningOrder': 'mean',
    'BaseWheel': 'mean'
}).reset_index()

# Add the 'Count' column to the aggregated DataFrame
df['Count'] = counts.values
df['Share'] = df['Count'] / df['Count'].sum()
df.sort_values(by=['Count'], ascending=False)

display(df.head(10))

df = co2.data.conditional_column_update(df, 'CategoryOf', 'm1', 'CategoryOf', 'M1')
df = co2.data.dummify_all_categoricals(df, dummy_columns=['Pool', 'FuelType', 'CategoryOf'], should_discretize_electricrange=False)

co2.data.save_processed_data(df, classification=False, pickle=False)

Incomplete rows dropped:473981


,CommercialName,FuelType,CategoryOf,InnovativeTechnology,Pool,Co2EmissionsWltp,EnginePower,ElectricConsumption,InnovativeEmissionsReductionWltp,ElectricRange,MassRunningOrder,BaseWheel,Count,Share
0,---,PETROL,M1,0,STELLANTIS,132.000000,96.0,0.0,0.000000,0.0,1323.000000,2605.0,1,1.058635e-07
1,108,PETROL,M1,0,STELLANTIS,110.441662,53.0,0.0,0.000000,0.0,921.120371,2340.0,30863,3.267265e-03
2,116 / / D,DIESEL,M1,0,BMW,120.000000,85.0,0.0,0.000000,0.0,1450.000000,2670.0,3,3.175905e-07
3,116 / / D,DIESEL,M1,1,BMW,119.500000,85.0,0.0,1.725000,0.0,1450.000000,2670.0,4,4.234540e-07
4,116 / / D AUT.,DIESEL,M1,0,BMW,121.000000,85.0,0.0,0.000000,0.0,1460.000000,2670.0,3,3.175905e-07
5,116 / / D AUT.,DIESEL,M1,1,BMW,120.600000,85.0,0.0,1.900000,0.0,1460.000000,2670.0,5,5.293175e-07
6,116 / / I,PETROL,M1,1,BMW,130.625000,80.0,0.0,1.800000,0.0,1395.000000,2670.0,8,8.469079e-07
7,116 / / I AUT.,PETROL,M1,1,BMW,136.000000,80.0,0.0,1.985714,0.0,1415.000000,2670.0,7,7.410445e-07
8,116 I,PETROL,M1,1,BMW,132.000000,80.0,0.0,2.100000,0.0,1850.000000,2670.0,1,1.058635e-07
9,116D,DIESEL,M1,0,BMW,121.660680,85.0,0.0,0.585485,0.0,1456.757282,2670.0,2060,2.180788e-04


In [7]:
co2.data.save_processed_data(df, classification=False, pickle=False)

Data saved to ../data/processed/co2_regression_20240115-201914


In [5]:
df.sort_values(by=['Count'], ascending=False)   

,CommercialName,InnovativeTechnology,Co2EmissionsWltp,EnginePower,ElectricConsumption,InnovativeEmissionsReductionWltp,ElectricRange,MassRunningOrder,BaseWheel,Count,...,FuelType_ETHANOL,FuelType_HYDROGEN,FuelType_LPG,FuelType_NATURALGAS,FuelType_NG,FuelType_PETROL,FuelType_PETROL/ELECTRIC,FuelType_UNKNOWN,CategoryOf_M1,CategoryOf_M1G
2352,CLIO,1,116.015614,66.192488,0.000000,1.630386,0.0,1181.066234,2582.949597,128922,...,0,0,0,0,0,1,0,0,1,0
6339,PANDA,0,122.401628,52.846083,0.000000,0.000480,0.0,1069.384510,2300.005183,108247,...,0,0,0,0,0,1,0,0,1,0
90,208,1,119.691755,69.166199,0.000000,1.827548,0.0,1133.583302,2540.000304,98623,...,0,0,0,0,0,1,0,0,1,0
1874,C3,1,125.206536,66.864953,0.000000,1.798329,0.0,1101.554765,2542.657380,93427,...,0,0,0,0,0,1,0,0,1,0
7833,T-ROC,1,147.736943,100.461553,0.000075,1.359641,0.0,1364.432126,2596.325762,93246,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,duster,0,140.000000,67.000000,0.000000,0.000000,0.0,1266.000000,2673.000000,1,...,0,0,0,0,0,1,0,0,1,0
9371,i 30,0,136.000000,81.000000,0.000000,0.000000,0.0,1331.000000,2650.000000,1,...,0,0,1,0,0,0,0,0,1,0
9408,octavia rs,0,28.000000,110.000000,0.000000,0.000000,0.0,1707.000000,2680.000000,1,...,0,0,0,0,0,0,1,0,1,0
71,2008,1,117.000000,81.000000,0.000000,1.800000,0.0,1730.000000,2605.000000,1,...,0,0,0,0,0,0,0,0,1,0
